In [8]:
import pandas as pd
import numpy as np
import pickle
import random
import os
import math
import sim_functions as sf
import sys
sys.path
sys.path.append('../models')
import state_params
from numpy.random import uniform
import TS_fancy
import eta

'''
Context: 
group_id,day_of_week,time_of_day,location,weather,previous_step_count,last_step_count,variation_in_step_counts
'''

A study needs some days. So you can specify days like this: 


In [3]:
days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=90),freq='30T')

I made 90 days because that's  the length of the study
However, because I am lazy, I'm only testing for a week

In [4]:
temp_indices = days[:24*7]
temp_indices

DatetimeIndex(['2015-07-22 00:00:00', '2015-07-22 00:30:00',
               '2015-07-22 01:00:00', '2015-07-22 01:30:00',
               '2015-07-22 02:00:00', '2015-07-22 02:30:00',
               '2015-07-22 03:00:00', '2015-07-22 03:30:00',
               '2015-07-22 04:00:00', '2015-07-22 04:30:00',
               ...
               '2015-07-25 07:00:00', '2015-07-25 07:30:00',
               '2015-07-25 08:00:00', '2015-07-25 08:30:00',
               '2015-07-25 09:00:00', '2015-07-25 09:30:00',
               '2015-07-25 10:00:00', '2015-07-25 10:30:00',
               '2015-07-25 11:00:00', '2015-07-25 11:30:00'],
              dtype='datetime64[ns]', length=168, freq='30T')

You can pass in a bunch of decision times. I just tested with one. 

In [5]:
decision_times = pd.DatetimeIndex(['2015-07-25 11:00:00'])
num_people = 2

Right now I think the number of people is being totally ignored. This needs to be fixed. 

In [6]:
def get_initial_context(num_people,first_index):
    return sf.get_initial_context(num_people,first_index)

In [17]:
peoples_context = get_initial_context(num_people,temp_indices[0])[0]

In [20]:
len(peoples_context)

9

In [15]:
def simulate_run(num_people,time_indices,decision_times,init=None,action_algorithm = None):
    
    
    initial_contexts = sf.get_initial_context(num_people,time_indices[0])
    
    ##for now number of people is one
    
    all_people = []
    
    
                 
    ##Initialize bandit algorithm
    
    
    all_rewards = []
    
    
    
    
    ##Time is constrained by the input times

    
    
    for n in range(num_people):
        #print('person')
        initial_context = initial_contexts[n]
    
        total_reward = 0
        
        the_eta = eta.eta()
    
        current_eta = the_eta.eta_init
        
        batch = [[] for i in range(len(time_indices))]
        
        ##place_holder
        Z_init = initial_context[:4]
        ##these two should be in context vector
        X_init = 0 
        I_init = (uniform() < 0.8)
        
         # policy initilization
        mu_beta = init.mu_2
        Sigma_beta = init.sigma_2
        
        initial_steps = sf.get_steps(initial_context,-1)
        current_steps = initial_steps
        action = -1 
        all_steps = []
    
        last_day = time_indices[0]
    
        new_day = False
    
        #for d in range(num_days):
    
    
        start_of_day = 0 
        end_of_day=0
        current_index=0
    
    
        first_week = time_indices[0].date()+pd.DateOffset(days=7)
        batch_index = 0 
        
        dt_counter = 0 
        
        
        
        
        for i in time_indices:
           
            
            ##Everything I do before I advance the day
            
            if i == time_indices[0]:
      
                Z_next = Z_init
                X_next = X_init
                I_next = I_init
      
    
            Z = Z_next
            X = X_next
            I = I_next
            
            
            if I==1:
                prob = TS_fancy.prob_cal(Z, X, mu_beta, Sigma_beta, init,current_eta)
                    # sample the action
                A = int(uniform() < prob)
                    
            else:
      
                prob = 0
                A = 0
            
            R = 1+Z[0]+(Z[0]**2)-(.05*X)+A*(1-.1*X)+np.random.normal(scale=1)
            
            

            
            
            if i.date()!=last_day.date():
           
                new_day=True
            
            
            
            decision_time = bool(i in decision_times)
    
            if i!=time_indices[0]:
          
                
                if i in decision_times:
                    
                    if action_algorithm==None:
                        action = sf.get_action(my_context,current_steps)
                    elif action_algorithm=='TS_fancy':
                        ##first get action, then update policy - so this weaving needs to be straight
                        action=A
                    else:
                        action = -1
                
                    dt_counter = dt_counter+1
           
                lsc = initial_context[sf.get_index('yesterday')]
                variation = initial_context[sf.get_index('variation')]
                if new_day:

                
                
        
                    if i<first_week:
                        variation = sf.get_variation_pre_week(variation,all_steps,time_indices,last_day)
                    else:
                        variation = sf.get_variation(all_steps,time_indices,last_day)
                
                
                    lsc = sf.get_new_lsc(all_steps[start_of_day:end_of_day])
           
                
            
                #which context is this? - do i need both?
                
                
                ##before getting action need to upate A and Prob

            
                ##fix this
                ##is this always what the reward should be?
                
                Z_next = initial_context[:4]
                X_next = TS_fancy.gen_nextdosage(X,A)
                I_next = (uniform()<.8)
        
        
               
            

            ##redo get_steps
            
                my_context = sf.get_context_revised(i,initial_context,current_steps,decision_time,lsc,variation,action)
               
                
                next_steps = sf.get_steps(my_context,action) 
                all_steps.append(next_steps)
                initial_context = my_context
                current_steps = next_steps
            else:
                if i in decision_times:
                    #print('type two decision time')
                    if action_algorithm==None:
                        action = sf.get_action(initial_context,current_steps)
                    elif action_algorithm=='TS_fancy':
                        ##first get action, then update policy - so this weaving needs to be straight
                        action=A
                    
                else:
                    action = -1
                dt_counter = dt_counter+1
                
                next_steps = sf.get_steps(initial_context,action) 
                all_steps.append(next_steps)
                current_steps = next_steps
                
                ##SET Z_next, X_next, I_next
                
                
                
            if new_day:
            
                start_of_day = current_index
                new_day=False
            
            
            batch[batch_index]=TS_fancy.make_batch(batch_index,Z,X,I,A,prob,R)
        
        
            total_reward = total_reward+R
            
            last_day = i
            end_of_day = current_index  
            current_index = current_index+1
           
            total_reward = total_reward+R
            
            
            ##this assumes we update the policy at the end
            if i in decision_times and dt_counter==4:
                temp = TS_fancy.policy_update_ts(batch[:batch_index], init)
                        #print(temp)
                mu_beta = temp[0]
                Sigma_beta = temp[1]
                eta_params = temp[2]
                the_eta.update_params(eta_params)
                current_eta = the_eta.eta_function
                dt_counter = 0
            batch_index = batch_index+1
        all_people.append(all_steps)
            
    return all_people


In [11]:
sp = state_params.state_params(10)

In [16]:
x=simulate_run(num_people,temp_indices,decision_times,init=sp,action_algorithm = 'TS_fancy')

In [95]:
all_runs = []
for i in range(5):
    all_runs.append(simulate_run(num_people,temp_indices,decision_times))